In [1]:
# Descargamos las librerias que necesitamos

import requests # Para hacer request
import re # Regex
from bs4 import BeautifulSoup # Para hacer la sopa del request

# Manejo de datos 

import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium

from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # para ignorar los warnings de python 
warnings.filterwarnings('ignore')

In [2]:
from selenium.webdriver.support.ui import WebDriverWait # proporciona la capacidad de esperar de manera explícita hasta que se cumpla una determinada condición antes de continuar con la ejecución del código
from selenium.webdriver.support import expected_conditions as EC # define una serie de condiciones esperadas que se pueden utilizar con WebDriverWait. Estas condiciones representan eventos o estados específicos que se esperan que ocurran en el navegador web, como la presencia de un elemento en la página, su visibilidad, etc. 
from selenium.webdriver.common.by import By # proporciona una serie de estrategias de localización para encontrar elementos en una página web.

In [3]:
opciones= Options() # se utiliza para configurar las opciones del navegador.
opciones.add_experimental_option('excludeSwitches', ['enable-automation']) #  se utiliza para agregar una opción experimental al navegador. En este caso, se está configurando la opción "excludeSwitches" con el valor ['enable-automation']. 
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False) # configuramos la opción "useAutomationExtension" con el valor False. Esta opción controla si se debe utilizar la extensión de automatización del navegador. Al establecerlo en False, se deshabilita la extensión de automatización.
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [24]:
espectaculos = []
urls_shows = []
precios = []

for i in range(1, 7):
    url = f'https://www.atrapalo.com/entradas/madrid/teatro-y-danza/monologos/p-{i}/'
    
    # hacemos la request a la página
    res1 = requests.get(url)
    # creamos la sopa
    sopa = BeautifulSoup(res1.content, 'html.parser')


    # Sacamos NOMBRES e iteramos para añadirlos a una lista
    nombres = sopa.find_all("h2", {"class": "clear nombre"})
    for i in nombres:
        espectaculos.append(i.getText().strip())
    
    # Sacamos urls de los shows e iteramos para añadirlos a una lista
    urls = sopa.find_all("div", {"data-url": True})
    for url in urls:
        urls_shows.append(url['data-url'])
    
    # Sacamos los precios de los shows e iteramos para añadirlos a una lista    
    p = sopa.find_all("span", {"class": "value"})
    
    for i in p:
        precios.append(i.getText().strip())

In [5]:
monologos_madrid = {"nombre":[], 
                        "precio": [],
                         "artista": [],
                        "lugar": [], 
                        "nota": [],
                   "n_opiniones": [],
                   "duracion": []}

# Introducimos los nombres de los shows al diccionario
    
monologos_madrid["nombre"] = espectaculos
    
# Introducimos los precios de los shows al diccionario

monologos_madrid["precio"] = precios

In [6]:
driver = webdriver.Chrome()
for url in urls_shows:
    driver.get(f"https://www.atrapalo.com{url}") 
    driver.implicitly_wait(5)

    # aceptamos las cookies de trapalo
    try:
        driver.find_element("css selector","#CybotCookiebotDialogBodyButtonAccept").click()
        sleep(2)
    except:
        pass
    
    # Capturamos nombre del artista
    try:
        monologos_madrid["artista"].append(driver.find_element("xpath", '/html/body/div[4]/div[1]/section/section[1]/div/div[1]/div[3]/a').text)
    except:
        monologos_madrid["artista"].append(np.nan)
    
    # Capturamos los lugar
    try:
        monologos_madrid["lugar"].append(driver.find_element("xpath", '//*[@id="widgetMapa"]/ul/li[2]/a').text)
    except:
        monologos_madrid["lugar"].append(np.nan)

    # Capturamos la nota
    try:
        monologos_madrid["nota"].append(driver.find_element("css selector", '#ranking-item-wrapper_0 > div.opinions-ranking__header > div > div.overall__body > div > span.overall__rating').text)
    except:
        monologos_madrid["nota"].append(np.nan)

    # Capturamos el número de opiniones
    try:
        monologos_madrid["n_opiniones"].append(driver.find_element("css selector", '#ranking-item-wrapper_0 > div.opinions-ranking__header > div > div.overall__body > div > span.overall__opinions > span:nth-child(1)').text)
    except:
        monologos_madrid["n_opiniones"].append(np.nan)

    # Capturamos la duración
    try:
        monologos_madrid["duracion"].append(driver.find_element("xpath", '/html/body/div[4]/div[1]/section/section[2]/div[1]/ul/div[1]/li[2]/p').text)
    except:
        monologos_madrid["duracion"].append(np.nan)

In [21]:
# Comprobamos la longitud de todas las llaves para comprobar que se ha descargado todos los datos con exito

print(len(monologos_madrid["nombre"]))
print(len(monologos_madrid["precio"]))
print(len(monologos_madrid["artista"]))
print(len(monologos_madrid["lugar"]))
print(len(monologos_madrid["nota"]))
print(len(monologos_madrid["n_opiniones"]))
print(len(monologos_madrid["duracion"]))

240
240
240
240
240
240
240


In [33]:
df_monologos_madrid = pd.DataFrame(monologos_madrid)
df_monologos_madrid

,nombre,precio,artista,lugar,nota,n_opiniones,duracion
0,La Hora y Media de El Club de la Comedia Madrid,22€,NaN,Teatro Príncipe Gran Vía,8.7,3930,NaN
1,Los Monólogos de El Golfo Comedy Club en Madrid,5€,Diego Daño,El Golfo Comedy Club,8.9,1656,Duración\nDe 70 a 120 minutos (según show)
2,"No cruces los brazos, David Cepo",17€,NaN,Teatro Arlequín Gran Vía,9.6,153,Duración\n75
3,La Noche de la Risa,18€,NaN,Teatro Capitol Gran Vía,9.9,1237,"Horario(s) evento\n20:00h, 22:30h"
4,Malasaña - Los Mejores Monólogos de Madrid,7€,Alex Salaberri Rubén Faura Marce González Raúl...,Estación Malasaña,9.9,1181,Duración\n1 hora
...,...,...,...,...,...,...,...
235,"Dos Gordos muy Grasiosos, en Estación Malasaña",7€,Javi Delpe,Estación Malasaña,9.4,29,Duración\n75 minutos (con descanso)
236,Luis Gerónimo Abreu - Camas,20€,NaN,Pequeño Teatro Gran Vía de Madrid,NaN,NaN,Duración\n75 minutos
237,Monólogos - El Remate Producciones en Estación...,6€,El Remate Producciones,Estación Malasaña,9.2,29,Duración\n60-90 minutos (dependiendo del show ...
238,Monólogos en San Sebastián de los Reyes,"6,50€",El Remate Producciones,La Nota Rock,NaN,NaN,Duración\n75 minutos (con descanso) aprox


In [34]:
# Especificar la ruta y el nombre del archivo CSV
ruta_archivo_csv = 'monologos_madrid.csv'

# Guardar el DataFrame en un archivo CSV
df_monologos_madrid.to_csv(ruta_archivo_csv, index=False)